In [1]:
import napari
from skimage import io
import btrack
import os

In [2]:
# take note of root directories on server and local equivalent for saving of dir structure locally
base_dir = server_rootdir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area'
# this is the root directory where files will be saved locally
local_rootdir = '/home/dayn/data/homuncu_loc_temp'
print('Here are the experimental data sets contained within base_dir:')
print(os.listdir(base_dir))

print('Here are the image data sets contained within the base directory:')
image_file_list = list()

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.tif'):
            image_file_list.append(os.path.join(root, file))

for image_file in image_file_list:
    folder_up = os.path.relpath(os.path.dirname(image_file), base_dir)
    file_name = os.path.basename(image_file)
    print(f'{folder_up}/{file_name}')

Here are the experimental data sets contained within base_dir:
['run2_23-02-104', 'run1_23-01-001_23-01-005']
Here are the image data sets contained within the base directory:
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1365.tif
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1363.tif
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1361.tif
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1362.tif
run2_23-02-104/48h pi/20230718_20X_23-02-104B2_Multichannel Z-Stack_20230718_1364.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1343.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1342.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1344.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multichannel Z-Stack_20230714_1341.tif
run2_23-02-104/2h pi/20230714_20X_23-02-104A4_Multicha

In [3]:
image_fn = image_file_list[2]

In [13]:
image = io.imread(image_fn)

In [4]:
# tracks_fn = image_fn.replace(server_rootdir, local_rootdir).replace('.tif', '_z_tracks_masks.h5')
with btrack.io.HDF5FileHandler('/home/dayn/data/homuncu_loc_temp/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314_z_tracks_masks.h5', 'r', obj_type='obj_type_1') as reader:
    segmentation = reader.segmentation
    tracks = reader.tracks

[INFO][2023/08/01 01:15:44 PM] Opening HDF file: /home/dayn/data/homuncu_loc_temp/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314_z_tracks_masks.h5...
[INFO][2023/08/01 01:15:51 PM] Loading segmentation (303, 2304, 2304)
[INFO][2023/08/01 01:15:51 PM] Loading tracks/obj_type_1
[INFO][2023/08/01 01:15:51 PM] Loading LBEP/obj_type_1
[INFO][2023/08/01 01:15:51 PM] Loading objects/obj_type_1 (12239, 5) (12239 filtered: None)
[INFO][2023/08/01 01:15:52 PM] Closing HDF file: /home/dayn/data/homuncu_loc_temp/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1314_z_tracks_masks.h5


In [7]:
import numpy.typing as npt

In [8]:
def color_by_mtb(track) -> npt.NDArray:
   # returns just the scalar value from each track, e.g.
   return track.properties["mtb_status"]

In [5]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='mtb_status')

In [6]:
recolored_segmentation2 = btrack.utils.update_segmentation(segmentation, tracks, color_by='mtb_area')

In [9]:
from btrack.constants import DEFAULT_EXPORT_PROPERTIES

In [10]:
keys = {k: i for i, k in enumerate(DEFAULT_EXPORT_PROPERTIES)}

keys.update(
    {
        key: idx
        for idx, key in enumerate(
            tracks[0].properties.keys(), start=max(keys.values()) + 1
        )
    }
)

In [11]:
keys

{'ID': 0,
 't': 1,
 'x': 2,
 'y': 3,
 'z': 4,
 'parent': 5,
 'root': 6,
 'state': 7,
 'generation': 8,
 'dummy': 9,
 'mtb_area': 10,
 'axis_minor_length': 11,
 'orientation': 12,
 'mtb_status': 13,
 'eccentricity': 14,
 'area': 15,
 'mean_intensity': 16,
 'class_id': 17,
 'axis_major_length': 18}

In [14]:
import numpy as np

In [12]:
len(keys)

19

In [15]:
coords_arr = np.concatenate(
    [
        track.to_array()[~np.array(track.dummy), : ].astype(int)
        for track in tracks
    ]
)


In [16]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,mtb_area,axis_minor_length,orientation,mtb_status,eccentricity,area,mean_intensity,class_id,axis_major_length
0,2,1,2234.346191,15.595745,0.0,2,2,5,0,False,"(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1,) array","(1, 4) array","(1,) array","(1,) array"


In [17]:
coords_arr[0]

array([   2,    1, 2234,   15,    0,    2,    2,    5,    0,    0,    0,
         32,   -1,    0,    0, 1363,  193,  108,  112,  114,    3,   66])

In [19]:
track = tracks[0]

In [20]:
segmentation.shape

(303, 2304, 2304)

In [21]:
t = 150

In [22]:
frame_coords = coords_arr[coords_arr[:, 1] == t]


In [24]:
len(frame_coords)

50

In [25]:
xc, yc = frame_coords[:, keys["x"]], frame_coords[:, keys["y"]]

In [35]:
xc

array([1236,   47,   41,  351,  415, 1158, 1113,  508, 1762, 1193, 1144,
       1778,   25,  218,   59,  277,  232, 1200, 1957, 1697, 1688, 1948,
       1096,  150,  171,  278, 1194,   16, 1214,    9, 1884,  220,   51,
        567, 1151, 1057, 1183, 1167, 1072,    9, 1637, 1074, 1381,  109,
        111, 1867, 1371,   32,  528, 2180])

In [28]:
color_by = 'mtb_area'

In [29]:
new_id = frame_coords[:, keys[color_by]]

In [34]:
new_id

array([93,  0, 57,  0,  3, 23,  0,  3,  5, 40,  0,  0,  0,  0, 19,  0,  0,
        0, 60,  0,  1,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  3,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0])

In [36]:
single_segmentation = segmentation[t]

In [41]:
from skimage.morphology import label

In [42]:
label(single_segmentation)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [53]:
from skimage.morphology import binary_erosion

In [60]:
from tqdm.auto import tqdm

In [61]:
eroded_segmentation = np.zeros_like(single_segmentation)
for segment_ID in tqdm(range(1, single_segmentation.max())):
    segment = single_segmentation == segment_ID
    # erode segment so that it doesnt touch neighbours
    eroded_segment = binary_erosion(segment, footprint=np.ones((5, 5)))
    
    # relabel segment semantically
    eroded_segmentation[eroded_segment] = segment_ID

  0%|          | 0/429 [00:00<?, ?it/s]

In [57]:
viewer.add_labels(eroded_segmentation)

<Labels layer 'eroded_segmentation' at 0x7f1a611287c0>

In [37]:
old_id = single_segmentation[yc, xc]

In [38]:
old_id

array([ 25, 156, 132, 413, 414,  30,  36, 404, 328,  66,  26, 337,  93,
       193, 140, 333, 153,   1, 364, 329, 321, 338,   5, 170, 145, 144,
       138, 105, 220,  84, 353, 133, 118, 103, 135,  38,  40,  45,  47,
        74, 331,  32, 255, 266, 298, 323, 240, 188, 409, 389],
      dtype=uint16)

In [56]:
viewer = napari.Viewer() 
viewer.add_image(single_segmentation)

<Image layer 'single_segmentation' at 0x7f198b7af430>

In [8]:
resized_segmentation = btrack.utils.update_segmentation(segmentation, tracks, scale = (1,0.5,0.5))

In [17]:
napari_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [7]:


viewer = napari.Viewer()

# viewer.add_image(image, channel_axis = -1)
viewer.add_labels(segmentation)
viewer.add_labels(recolored_segmentation)
viewer.add_labels(recolored_segmentation2)
# viewer.add_labels(resized_segmentation)
viewer.add_tracks(napari_tracks, properties=properties)

/home/dayn/analysis/btrack/btrack/dataio.py:3: UserWarning: `btrack.dataio` has been deprecated. Please use `btrack.io` subpackage instead.
  warnings.warn(  # noqa: B028


Assistant skips harvesting pyclesperanto as it's not installed.


NameError: name 'napari_tracks' is not defined

In [16]:
viewer.add_labels(recolored_segmentation)

<Labels layer 'recolored_segmentation [2]' at 0x7fe39b027d60>

In [11]:
import macrohet

In [12]:
macrohet.notify.send_sms('proof loaded')

AttributeError: module 'macrohet' has no attribute 'notify'